In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import random
import time
from datetime import date, timedelta
from pandas import DataFrame,Series

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor


from dfply import *

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
start_time = time.time()
df1 = pd.read_csv('gs://studentsmall/sep000000000000.csv')
df2 = pd.read_csv('gs://studentsmall/sep000000000001.csv')
df3 = pd.read_csv('gs://studentsmall/sep000000000002.csv')
df4 = pd.read_csv('gs://studentsmall/sep000000000003.csv')
df5 = pd.read_csv('gs://studentsmall/sep000000000004.csv')
df6 = pd.read_csv('gs://studentsmall/sep000000000005.csv')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
df = pd.DataFrame(np.concatenate([df1.values, df2.values, df3.values, df4.values, df5.values ,df6.values]), columns=df1.columns)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
len(df)

In [ ]:
total_users = df['userID'].nunique()
total_users

In [ ]:
data = df.copy()

In [ ]:
data.info()

In [ ]:
# caculate userAge, which is how many days the event happened since gaUserStartDate
start_time = time.time()
data['eventTimestamp'] = pd.to_datetime(data['eventTimestamp'])
data['gaUserStartDate'] = pd.to_datetime(data['gaUserStartDate'])
data['userAge'] = (data['eventTimestamp'] - data['gaUserStartDate']).dt.days
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
userID_uni = data['userID'].unique()
userID_uni = pd.DataFrame(userID_uni, columns = ['userID'])


In [ ]:
start_time = time.time()
total_result = {}
day_cases = [3, 7, 20, 30]

for day_case in day_cases:
    day = data[(data['userAge'] <= day_case) & (data['gaUserStartDate'] >= min(data['eventTimestamp']))]
    print(len(day), day_case)

    # countAdImpression: select 'adImpression' from eventName, and then count rows grouping by userID
    impression_day = day[day['eventName'] == 'adImpression'][['userID', 'eventName','action']]
    userID_impression_day = impression_day.groupby('userID').size()
    userID_impression_day = pd.DataFrame(userID_impression_day,columns = ['countAdImpression'])

    # countAdClicked: select 'adImpression' from eventName and 'clicked' from action, then count rows grouping by userID
    impression_click_day = day[(day['eventName'] == 'adImpression') & (day['action'] == 'clicked')][['userID', 'eventName','action']]
    userID_impression_click_day = impression_click_day.groupby('userID').size()
    userID_impression_click_day = pd.DataFrame(userID_impression_click_day,columns = ['countAdClicked'])

    # missionTime: select 'missionCompleted' from eventName, then sum time
    missionTime_raw_day = day[(day['eventName'] == 'missionCompleted')][['userID', 'time']]
    missionTime_day = missionTime_raw_day.groupby('userID').sum()
    missionTime_day = missionTime_day.rename(columns = {'time': 'missionTime'})

    # CountMission = select 'missionCompleted'from eventName, then count rows
    missionCount_day = missionTime_raw_day.groupby('userID').size()
    missionCount_day = pd.DataFrame(missionCount_day, columns=['CountMission'])
    
    # timePerMission
    userID_timePerMission_day = pd.merge(missionTime_day,missionCount_day, on='userID')
    userID_timePerMission_day['timePerMission']= userID_timePerMission_day['missionTime']/userID_timePerMission_day['CountMission']
    
    # dailyMissionTime = select "dailyMissionCompleted" from eventName, then sum time
    dailyMissionTime_raw_day = day[(day['eventName'] == 'dailyMissionCompleted')][['userID', 'time']]
    dailyMissionTime_day = dailyMissionTime_raw_day.groupby('userID').sum()
    dailyMissionTime_day = dailyMissionTime_day.rename(columns = {'time': 'dailyMissionTime'})
    
    # CountDailyMission = select "dailyMissionCompleted" from eventName, then count rows
    dailyMissionCount_day = dailyMissionTime_raw_day.groupby('userID').size()
    dailyMissionCount_day = pd.DataFrame(dailyMissionCount_day, columns=['CountDailyMission'])
    
    # timePerDailyMission
    timePerDailyMission_day = pd.merge(dailyMissionTime_day,dailyMissionCount_day, on='userID')
    timePerDailyMission_day['timePerDailyMission']= dailyMissionTime_day['dailyMissionTime']/dailyMissionCount_day['CountDailyMission']
     
    # totalHintsUsed: select 'missionCompleted' or 'dailyMissionCompleted' from eventName, then sum hintUsed
    hint_day = day[(day['eventName'] == 'missionCompleted')][['userID', 'hintsUsed']]
    totalHint_day = hint_day.groupby('userID').sum()

    # hintPerMission: totalHintsUsed/CountMission
    hintPerMission_day = pd.merge(totalHint_day,missionCount_day, on='userID')
    hintPerMission_day['hintsPerMission'] = hintPerMission_day['hintsUsed']/hintPerMission_day['CountMission']
    hintPerMission_day = hintPerMission_day.drop(columns = ['CountMission'])
    
    # Level reached at end of the day case
    level_day = day.groupby(['userID'])['userLevel'].max()
    level_day = pd.DataFrame(level_day, columns = ['userLevel'])
    
    # hintPerLevel
    hintPerLevel_day = pd.merge(totalHint_day,level_day, on='userID')
    hintPerLevel_day['hintsPerLevel'] = hintPerLevel_day['hintsUsed']/hintPerLevel_day['userLevel']
    hintPerLevel_day = hintPerLevel_day.drop(columns = ['hintsUsed'])
    
    # countSpecialOffer: select 'specialOffer' from eventName, then count rows
    speOffer_day = day[(day['eventName'] == 'specialOffer')]
    speoffer_count_day = speOffer_day.groupby('userID').size()
    speoffer_count_day = pd.DataFrame(speoffer_count_day,columns = ['countSpeOffer'])

    # countSpecialOfferAccept: select 'specialOffer' from eventName and 'accepted' from action, and then count rows 
    SpeOffer_accept_day = day[(day['eventName'] == 'specialOffer') & (day['action'] == 'accepted')]
    SpeOffer_accept_day = SpeOffer_accept_day.groupby('userID').size()
    SpeOffer_accept_day = pd.DataFrame(SpeOffer_accept_day,columns = ['countSpeOfferAccept'])
    
    # totalDuplicateWords: select 'missionCompleted' or 'dailyMissionCompleted' from eventName, and then sum duplicateWords
    # grouping by userID
    DuplicateWords_raw = day[day['eventName'] == 'missionCompleted'][['userID','duplicateWords']]
    DuplicateWords = DuplicateWords_raw.groupby('userID').sum()

    # duplicateWordsPerMis: duplicateWords/CountMission
    DuplicateWordsPerMis = pd.merge(DuplicateWords,missionCount_day, on='userID')
    DuplicateWordsPerMis['duplicateWordsPerMis'] = DuplicateWordsPerMis['duplicateWords']/DuplicateWordsPerMis['CountMission']
    DuplicateWordsPerMis = DuplicateWordsPerMis.drop(columns = ['CountMission'])
    
    # totalExtraWords: select 'missionCompleted' from eventName, and then sum extraWords
    # grouping by userID
    ExtraWords_raw = day[day['eventName'] == 'missionCompleted'][['userID','extraWords']]
    ExtraWords = ExtraWords_raw.groupby('userID').sum()

    # extraWordsPerMis: totalExtraWords/CountMission
    extraWordsPerMis = pd.merge(ExtraWords,missionCount_day, on='userID')
    extraWordsPerMis['extraWordsPerMis'] = extraWordsPerMis['extraWords']/extraWordsPerMis['CountMission']
    extraWordsPerMis = extraWordsPerMis.drop(columns = ['CountMission'])
    
    # totalInvalidWords: select 'missionCompleted' from eventName, and then sum invalidWords
    InvalidWords_raw = day[day['eventName'] == 'missionCompleted'][['userID','invalidWords']]
    InvalidWords = InvalidWords_raw.groupby('userID').sum()

    # invalidWordsPerMis
    InvalidWordsPerMis = pd.merge(InvalidWords,missionCount_day, on='userID')
    InvalidWordsPerMis['invalidWordsPerMis'] = InvalidWordsPerMis['invalidWords']/InvalidWordsPerMis['CountMission']
    InvalidWordsPerMis = InvalidWordsPerMis.drop(columns = ['CountMission'])
    
    # playerIQ: select 'missionCompleted' or 'dailyMissionCompleted' from eventName, and then sum playerIQ
    # grouping by userID
    playerIQ_raw = day[(day['eventName'] == 'missionCompleted')|(day['eventName'] == 'dailyMissionCompleted')][['userID','playerIQ']]
    playerIQ = playerIQ_raw.groupby('userID').sum()

    
    # push: select out "gameStarted" from eventName, find the push status when eventTimestamp = max(eventTimestamp) 
    # grouping by each user
    push_raw = day[day['eventName'] == 'gameStarted'][['userID','push','eventTimestamp']]
    push = push_raw.loc[push_raw.groupby("userID")["eventTimestamp"].idxmax()]
    push = push.drop(columns = ['eventTimestamp']).set_index('userID') 
    
    # total shuffles: select missionCompleted and sum shuffles
    # shuffle per mission: total shuffles/CountMission
    totalShuffles = day[(day['eventName']=='missionCompleted')].groupby(['userID']).agg({'missionName':'nunique','shuffles':'sum'})
    totalShuffles['shufflesPerMission']=totalShuffles['shuffles'].div(totalShuffles['missionName'])
    shuffles = totalShuffles.drop(columns = ['missionName'])
    
    
    # building userprofile through acquisitionchannel,platform,
    user_profile=(day >> select('userID','gaUserAcquisitionChannel','platform') >> group_by('userID')).drop_duplicates()
    user_profile = user_profile[user_profile.gaUserAcquisitionChannel != "None"]
    user_profile = user_profile.drop_duplicates(subset='userID', keep="first").set_index('userID')
    
    # user removedAds 
    user_removedAds_raw = day[day['eventName'] == 'gameStarted'][['userID','removedAds','eventTimestamp']]
    user_removedAds = user_removedAds_raw.loc[user_removedAds_raw.groupby("userID")["eventTimestamp"].idxmax()]
    user_removedAds = user_removedAds.drop(columns = ['eventTimestamp']).set_index('userID') 
    
    # countVideoOffer: select 'videoOffer' from eventName and corresponding action, then count rows
    vidOffer_cvv = day[(day['eventName'] == 'videoOffer') & (day['action'] == 'CVV')]
    vidOffer_cvv_count = vidOffer_cvv.groupby('userID').size()
    vidOffer_cvv_count = pd.DataFrame(vidOffer_cvv_count, columns = ['countCVV'])
    
    vidOffer_no = day[(day['eventName'] == 'videoOffer') & (day['action'] == 'NO')]
    vidOffer_no_count = vidOffer_no.groupby('userID').size()
    vidOffer_no_count = pd.DataFrame(vidOffer_no_count, columns = ['countNO'])
    
    vidOffer_noCmplt = day[(day['eventName'] == 'videoOffer') & (day['action'] == 'NOTCOMPLETED')]
    vidOffer_noCmplt_count = vidOffer_noCmplt.groupby('userID').size()
    vidOffer_noCmplt_count = pd.DataFrame(vidOffer_noCmplt_count, columns = ['countNoCmplt'])
    
    # lifespan
    user_lifespan=(day >> select('userID','userAge') >> group_by('userID') >> summarize(Lifespan=X.userAge.max()))
    
    # adRevenue for ad impressions and video offers
    total_adRevenue = (day >> select('userID','almaxRevenue') >> group_by('userID') >> summarize(ad_Rev=X.almaxRevenue.sum()))
    adImpression_revenue = (day >> select('userID','almaxRevenue', 'almaxAdUnitType') >>  mask(X.almaxAdUnitType == 'BANNER') >> group_by('userID') >> summarize(adImpression_Rev=X.almaxRevenue.sum()))
    videoOffer_revenue = (day >> select('userID','almaxRevenue', 'almaxAdUnitType') >>  mask((X.almaxAdUnitType == 'INTER') |  (X.almaxAdUnitType == 'REWARD')) >> group_by('userID') >> summarize(videoOffer_Rev=X.almaxRevenue.sum()))

    # merge into one dataframe
    agg_data = pd.merge(userID_uni,userID_impression_day,on='userID', how='left')
    agg_data = pd.merge(agg_data, userID_impression_click_day, on='userID', how='left')
    agg_data = pd.merge(agg_data, userID_timePerMission_day, on='userID', how='left')
    agg_data = pd.merge(agg_data, timePerDailyMission_day, on='userID', how= 'left')
    agg_data = pd.merge(agg_data, hintPerMission_day, on='userID', how='left')
    agg_data = pd.merge(agg_data, hintPerLevel_day, on= 'userID', how='left')
    agg_data = pd.merge(agg_data, speoffer_count_day, on='userID', how='left')
    agg_data = pd.merge(agg_data, SpeOffer_accept_day, on='userID', how='left')
    agg_data = pd.merge(agg_data, DuplicateWordsPerMis, on='userID', how='left')
    agg_data = pd.merge(agg_data, extraWordsPerMis, on='userID', how='left')
    agg_data = pd.merge(agg_data, InvalidWordsPerMis, on='userID', how='left')
    agg_data = pd.merge(agg_data, playerIQ, on='userID', how='left')
    agg_data = pd.merge(agg_data, push, on='userID', how='left')
    agg_data = pd.merge(agg_data, shuffles, on='userID', how='left')
    agg_data = pd.merge(agg_data, user_profile, on='userID', how='left')
    agg_data = pd.merge(agg_data, user_removedAds, on='userID', how='left')
    agg_data = pd.merge(agg_data, vidOffer_cvv_count, on='userID', how='left')
    agg_data = pd.merge(agg_data, vidOffer_no_count, on='userID', how='left')
    agg_data = pd.merge(agg_data, vidOffer_noCmplt_count, on='userID', how='left')
    agg_data = pd.merge(agg_data, user_lifespan, on='userID', how='left')
    agg_data = pd.merge(agg_data, total_adRevenue, on='userID', how='left')
    agg_data = pd.merge(agg_data, adImpression_revenue, on='userID', how='left')
    agg_data = pd.merge(agg_data, videoOffer_revenue, on='userID', how='left')
    
    total_result[day_case] = agg_data
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
total_result[3].head()
## most of nulls could be filled with 0

In [ ]:
# the final dataframe for data from day0 to day3 after dropping rows with NaN in all columns
var = total_result[3]
var.set_index('userID', inplace = True)
var.dropna(axis=0, how='all', thresh=None, subset=None, inplace=True)

In [ ]:
var = var.dropna(subset = ['gaUserAcquisitionChannel', 'platform'])
var.fillna(0 , inplace = True)
var.head(2)

In [ ]:
userNoRev = var[(var['removedAds']==0) & (var['ad_Rev'] == 0)].index
var.drop(userNoRev, inplace = True)

In [ ]:
var_heatmap = var.drop(columns = ['gaUserAcquisitionChannel', 'platform'])
f, ax = plt.subplots(figsize=(20, 16))
corr = var_heatmap.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax,annot=True)

In [ ]:
kmeans_var = var.drop(columns = ['gaUserAcquisitionChannel', 'platform'])
removeAds_players = kmeans_var[kmeans_var['removedAds']==1]
#kmeans_var["Ad_Impression_Value"] = 4*(kmeans_var['countAdImpression'] - kmeans_var['countAdClicked']) + 10*kmeans_var['countAdClicked']
kmeans_var = kmeans_var.drop(columns = ['countAdImpression', 'countAdClicked','timePerMission','timePerDailyMission','hintsPerMission',
                                        'timePerDailyMission','hintsPerLevel','duplicateWordsPerMis','extraWordsPerMis','invalidWordsPerMis',
                                      'shufflesPerMission','removedAds', 'countCVV', 'countNO', 'countNoCmplt', 'ad_Rev', 'adImpression_Rev', 'videoOffer_Rev'])

In [ ]:
kmeans_var = kmeans_var[kmeans_var.userLevel != 0]
kmeans_var.to_csv('kmeans_var.csv')
kmeans_var_array = kmeans_var.to_numpy()
scaler = MinMaxScaler()
kmeans_var_array = scaler.fit_transform(kmeans_var_array)

In [ ]:
kmeans_var_array

In [ ]:
kmeans_var.info()

In [ ]:
Error =[]
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i).fit(kmeans_var_array)
    kmeans.fit(kmeans_var_array)
    Error.append(kmeans.inertia_)

    
fig = plt.figure()
plt.plot(range(1, 11), Error)
plt.title('Elbow method')
plt.xlabel('No of clusters')
plt.ylabel('Error')
plt.show()
fig.savefig("ElbowCurve.jpg")

In [ ]:
# build k-means clustering model with 4 clusters
kmeans4 = KMeans(n_clusters=4,random_state=40)
y_kmeans4 = kmeans4.fit_predict(kmeans_var_array)
print(y_kmeans4)

kmeans4.cluster_centers_

In [ ]:
for i in range(0,len(kmeans_var.columns)):
    for n in range(0,4):
        print(kmeans_var.columns[i], ", cluster: ", n)
        print("Center: ", kmeans4.cluster_centers_[n][i])
        print('----------------------------------------')

In [ ]:
for i in range(0,len(kmeans_var.columns)):
    print(str(kmeans_var.columns[i]) + ' ' +str(kmeans4.cluster_centers_[1][i]))

In [ ]:
kmeans_var.columns

In [ ]:
# plot the 4 clusters
for i in range(0,len(kmeans_var.columns)):
    plt.scatter(
        kmeans_var_array[y_kmeans4 == 0, i], kmeans_var_array[y_kmeans4 == 0, 1],
        s=50, c='lightgreen',
        marker='s', edgecolor='black',
        label='cluster 0'
    )

    plt.scatter(
        kmeans_var_array[y_kmeans4 == 1, i], kmeans_var_array[y_kmeans4 == 1, 1],
        s=50, c='orange',
        marker='o', edgecolor='black',
        label='cluster 1'
    )

    plt.scatter(
        kmeans_var_array[y_kmeans4 == 2, i], kmeans_var_array[y_kmeans4 == 2, 1],
        s=50, c='lightblue',
        marker='v', edgecolor='black',
        label='cluster 2'
    )

    plt.scatter(
        kmeans_var_array[y_kmeans4 == 3, i], kmeans_var_array[y_kmeans4 == 3, 1],
        s=50, c='red',
        marker='.', edgecolor='black',
        label='cluster 3'
    )

#    plot the centroids
    plt.scatter(
        kmeans4.cluster_centers_[:, i], kmeans4.cluster_centers_[:, 1],
        s=250, marker='*',
        c='red', edgecolor='black',
        label='centroids'
    )
    plt.legend(scatterpoints=1)
    plt.xlabel(kmeans_var.columns[i])
    plt.ylabel("countMissions")
    plt.grid()
    plt.show()

In [ ]:
# for i in range(0,len(kmeans_var.columns)):
fig = plt.figure(figsize=(15, 10))
plt.scatter(
    kmeans_var_array[y_kmeans4 == 0, 0], kmeans_var_array[y_kmeans4 == 0, 1],
    s=200, c='lightgreen',
    marker='s', edgecolor='black',
    label='cluster 0'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 1, 0], kmeans_var_array[y_kmeans4 == 1, 1],
    s=200, c='orange',
    marker='o', edgecolor='black',
    label='cluster 1'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 2, 0], kmeans_var_array[y_kmeans4 == 2, 1],
    s=200, c='lightblue',
    marker='v', edgecolor='black',
    label='cluster 2'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 3, 0], kmeans_var_array[y_kmeans4 == 3, 1],
    s=200, c='red',
    marker='.', edgecolor='black',
    label='cluster 3'
)

#    plot the centroids
plt.scatter(
    kmeans4.cluster_centers_[:, 0], kmeans4.cluster_centers_[:, 1],
    s=700, marker='*',
    c='red', edgecolor='black',
    label='centroids'
)
plt.legend(scatterpoints=1,fontsize=20)
plt.xlabel(kmeans_var.columns[0],fontsize=20)
plt.ylabel("Total Missions",fontsize=20)
plt.title("Total Missions vs Total Mission Time", fontsize=30)
plt.xticks(fontsize=20, rotation=0)
plt.yticks(fontsize=20)
plt.grid()
plt.show()
fig.savefig("CountMissions_MissionTime.jpg")

In [ ]:
fig = plt.figure(figsize=(20, 16))
plt.scatter(
    kmeans_var_array[y_kmeans4 == 0, 2], kmeans_var_array[y_kmeans4 == 0, 1],
    s=200, c='lightgreen',
    marker='s', edgecolor='black',
    label='cluster 0'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 1, 2], kmeans_var_array[y_kmeans4 == 1, 1],
    s=200, c='orange',
    marker='o', edgecolor='black',
    label='cluster 1'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 2, 2], kmeans_var_array[y_kmeans4 == 2, 1],
    s=200, c='lightblue',
    marker='v', edgecolor='black',
    label='cluster 2'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 3, 2], kmeans_var_array[y_kmeans4 == 3, 1],
    s=200, c='yellow',
    marker='.', edgecolor='black',
    label='cluster 3'
)

#    plot the centroids
plt.scatter(
    kmeans4.cluster_centers_[:, 2], kmeans4.cluster_centers_[:, 1],
    s=3000, marker='*',
    c='red', edgecolor='black',
    label='centroids'
)

    
plt.legend(scatterpoints=1,fontsize=20)
plt.xlabel(kmeans_var.columns[2],fontsize=20)
plt.ylabel("Total Missions",fontsize=20)
plt.title("Total Missions vs Total Daily Mission Time", fontsize=30)
plt.xticks(fontsize=20, rotation=0)
plt.yticks(fontsize=20)
plt.grid()
plt.show()
fig.savefig("CountMissions_DailyMissionTime.jpg")

In [ ]:
fig = plt.figure(figsize=(20, 16))
plt.scatter(
    kmeans_var_array[y_kmeans4 == 0, 4], kmeans_var_array[y_kmeans4 == 0, 1],
    s=200, c='lightgreen',
    marker='s', edgecolor='black',
    label='cluster 0'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 1, 4], kmeans_var_array[y_kmeans4 == 1, 1],
    s=200, c='orange',
    marker='o', edgecolor='black',
    label='cluster 1'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 2, 4], kmeans_var_array[y_kmeans4 == 2, 1],
    s=200, c='lightblue',
    marker='v', edgecolor='black',
    label='cluster 2'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 3, 4], kmeans_var_array[y_kmeans4 == 3, 1],
    s=200, c='red',
    marker='.', edgecolor='black',
    label='cluster 3'
)

#    plot the centroids
plt.scatter(
    kmeans4.cluster_centers_[:, 4], kmeans4.cluster_centers_[:, 1],
    s=700, marker='*',
    c='red', edgecolor='black',
    label='centroids'
)
plt.legend(scatterpoints=1,fontsize=20)
plt.xlabel(kmeans_var.columns[4],fontsize=20)
plt.ylabel("Total Missions",fontsize=20)
plt.title("Total Missions vs. Total Hint Used", fontsize=30)
plt.xticks(fontsize=20, rotation=0)
plt.yticks(fontsize=20)
plt.grid()
plt.show()
fig.savefig("CountMissions_TotalHint.jpg")

In [ ]:
fig = plt.figure(figsize=(20, 16))
plt.scatter(
    kmeans_var_array[y_kmeans4 == 0, 8], kmeans_var_array[y_kmeans4 == 0, 1],
    s=200, c='lightgreen',
    marker='s', edgecolor='black',
    label='cluster 0'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 1, 8], kmeans_var_array[y_kmeans4 == 1, 1],
    s=200, c='orange',
    marker='o', edgecolor='black',
    label='cluster 1'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 2, 8], kmeans_var_array[y_kmeans4 == 2, 1],
    s=200, c='lightblue',
    marker='v', edgecolor='black',
    label='cluster 2'
)

plt.scatter(
    kmeans_var_array[y_kmeans4 == 3, 8], kmeans_var_array[y_kmeans4 == 3, 1],
    s=200, c='red',
    marker='.', edgecolor='black',
    label='cluster 3'
)

#    plot the centroids
plt.scatter(
    kmeans4.cluster_centers_[:, 8], kmeans4.cluster_centers_[:, 1],
    s=700, marker='*',
    c='red', edgecolor='black',
    label='centroids'
)
plt.legend(scatterpoints=1,fontsize=20)
plt.xlabel(kmeans_var.columns[8],fontsize=20)
plt.ylabel("Total Missions",fontsize=20)
plt.title("Total Missions vs Total Duplicated Words", fontsize=30)
plt.xticks(fontsize=20, rotation=0)
plt.yticks(fontsize=20)
plt.grid()
plt.show()

In [ ]:
kmeans_var['cluster'] = y_kmeans4

In [ ]:
np.shape(kmeans_var[(kmeans_var['cluster'] == 0)])

In [ ]:
np.shape(kmeans_var[(kmeans_var['cluster'] == 1)])

In [ ]:
np.shape(kmeans_var[(kmeans_var['cluster'] == 2)])

In [ ]:
np.shape(kmeans_var[(kmeans_var['cluster'] == 3)])

In [ ]:
kmeans_var.head(2)

In [ ]:
removeAds_cluster = kmeans_var.loc[kmeans_var.index.intersection(set(removeAds_players.index))]
removeAds_cluster.head(2)



In [ ]:
fig = plt.figure(figsize=(20, 16))
removeAds_cluster['cluster'].value_counts().sort_index().plot(kind = 'bar');
plt.xlabel("Cluster", fontsize=30)
plt.ylabel("Total number of players", fontsize=30)
plt.title("Number of Ad Removal Purchasers by Cluster", fontsize=40)
plt.xticks(fontsize=30, rotation=0)
plt.yticks(fontsize=30)
fig.savefig("Num_players_bar.jpg")

In [ ]:
cluster0 = var.loc[kmeans_var[(kmeans_var['cluster'] == 0)].index,:]
cluster1 = var.loc[kmeans_var[(kmeans_var['cluster'] == 1)].index,:]
cluster2 = var.loc[kmeans_var[(kmeans_var['cluster'] == 2)].index,:]
cluster3 = var.loc[kmeans_var[(kmeans_var['cluster'] == 3)].index,:]

In [ ]:

Day20_adRev=total_result[30][['userID','ad_Rev']]
kmeans_var['userID']=kmeans_var.index
cluster_info=kmeans_var[['userID','cluster']]
cluster_revenue=pd.merge(kmeans_var['cluster'],Day20_adRev, on='userID',how='left')

fig = plt.figure()
cluster_revenue.boxplot(by ='cluster', column =['ad_Rev'], figsize=(12,8)) 
plt.xlabel("Cluster", fontsize=15)
plt.ylabel("Ad Revenue", fontsize=15)
plt.title("Ad Revenue by Cluster", fontsize=20)
plt.xticks(fontsize=15, rotation=0)
plt.yticks(fontsize=15)
fig.savefig("AdRev_boxplot.jpg")

In [ ]:
Day20_adRev=total_result[7][['userID','ad_Rev']]
kmeans_var['userID']=kmeans_var.index
cluster_info=kmeans_var[['userID','cluster']]
cluster_revenue=pd.merge(kmeans_var['cluster'],Day20_adRev, on='userID',how='left')
cluster_revenue.boxplot(by ='cluster', column =['ad_Rev'],  figsize=(12,8)) 

In [ ]:
desired_indices0 = [i for i in cluster0.index if i not in removeAds_players.index]
cluster0.loc[desired_indices0,:][['Lifespan','userLevel','CountMission','ad_Rev', 'adImpression_Rev', 'videoOffer_Rev','dailyMissionTime']].describe()

In [ ]:
desired_indices1 = [i for i in cluster1.index if i not in removeAds_players.index]
cluster1.loc[desired_indices1,:][['Lifespan','userLevel','CountMission','ad_Rev', 'adImpression_Rev', 'videoOffer_Rev','dailyMissionTime']].describe()

In [ ]:
desired_indices2 = [i for i in cluster2.index if i not in removeAds_players.index]
cluster2.loc[desired_indices2,:][['userLevel','CountMission','ad_Rev', 'adImpression_Rev', 'videoOffer_Rev','dailyMissionTime']].describe()

In [ ]:
desired_indices3 = [i for i in cluster3.index if i not in removeAds_players.index]
cluster3.loc[desired_indices3,:][['userLevel','CountMission','ad_Rev', 'adImpression_Rev', 'videoOffer_Rev','dailyMissionTime']].describe()

## Summarize each cluster using data from day 0 to day 30

In [ ]:
day30 = total_result[30]
day30.set_index('userID', inplace = True)
day30.dropna(axis=0, how='all', thresh=None, subset=None, inplace=True)

day30 = day30.dropna(subset = ['gaUserAcquisitionChannel', 'platform'])
day30.fillna(0 , inplace = True)

In [ ]:
day30.head(2)

In [ ]:
day30_cluster0= day30.loc[cluster0.index,:]
day30_cluster1= day30.loc[cluster1.index,:]
day30_cluster2= day30.loc[cluster2.index,:]
day30_cluster3= day30.loc[cluster3.index,:]

In [ ]:
day30_cluster0['ad_Rev'].describe()

In [ ]:
day30_cluster1['ad_Rev'].describe()

In [ ]:
day30_cluster2['ad_Rev'].describe()

In [ ]:
day30_cluster3['ad_Rev'].describe()

## Lasso Regression within each cluster to predict total ads value

### Cluster 0

In [ ]:
cols = ['countAdImpression','countAdClicked','timePerMission', 'timePerDailyMission', 'hintsPerMission','dailyMissionTime', 'hintsPerLevel','duplicateWordsPerMis','extraWordsPerMis', 'invalidWordsPerMis','shufflesPerMission','countCVV', 'countNoCmplt', 'ad_Rev', 'adImpression_Rev', 'videoOffer_Rev']

cls0_x = day7_cluster0.drop(columns=cols)
cls0_x = pd.get_dummies(cls0_x, columns = ['gaUserAcquisitionChannel','platform'], prefix=['Chan', 'Plat'], drop_first=False)
cls0_x = cls0_x.drop(columns='Plat_IOS')
cls0_x = cls0_x.drop(columns = ['extraWords', 'invalidWords'])

var_heatmap0 = cls0_x
f, ax = plt.subplots(figsize=(30, 25))
corr = var_heatmap0.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax,annot=True)

In [ ]:
# LassoCV on cluster 0
cls0_x = preprocessing.normalize(cls0_x, norm='l2')
cls0_y = day7_cluster0['ad_Rev'] 

x0_train, x0_test, y0_train, y0_test = train_test_split(cls0_x, cls0_y, test_size=0.2, random_state=42)
reg_lcv = LassoCV(cv=5, random_state=0)
cls0_reg_lcv = reg_lcv.fit(x0_train, y0_train)

y0_test_predicted_lcv = cls0_reg_lcv.predict(x0_test)

print('R-square: ', cls0_reg_lcv.score(x0_test, y0_test))
print('MSE: ', mean_squared_error(y0_test,y0_test_predicted_lcv))
print('Coeffcients: ', cls0_reg_lcv.coef_)

In [ ]:
alpha = np.arange(0.1, 1, 0.01)
parameters = {'alpha': alpha}

lml = linear_model.Lasso(normalize=False)

clf = GridSearchCV(lml, parameters)
print(clf.fit(x0_train, y0_train).best_params_['alpha'])
cls0_lml = linear_model.Lasso(alpha = clf.fit(x0_train, y0_train).best_params_['alpha']).fit(x0_train, y0_train)


y0_test_predicted_lml = clf.predict(x0_test)
print(cls0_lml.score(x0_test, y0_test))
print(mean_squared_error(y0_test,y0_test_predicted_lml))

### Cluster 1

In [ ]:
cls1_x = day7_cluster1.drop(columns=cols)
cls1_x = pd.get_dummies(cls1_x, columns = ['gaUserAcquisitionChannel','platform'], prefix=['Chan', 'Plat'], drop_first=False)
cls1_x = cls1_x.drop(columns = ['Plat_IOS','extraWords','invalidWords'])

var_heatmap1 = cls1_x
f, ax = plt.subplots(figsize=(20, 16))
corr = var_heatmap1.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax,annot=True)

In [ ]:
# LassoCV on cluster 1
cls1_x = preprocessing.normalize(cls1_x, norm='l2')
cls1_y = day7_cluster1['ad_Rev'] 

x1_train, x1_test, y1_train, y1_test = train_test_split(cls1_x, cls1_y, test_size=0.2, random_state=42)

reg_lcv = LassoCV(cv=5, random_state=0)
cls1_reg_lcv = reg_lcv.fit(x1_train, y1_train)

y1_test_predicted_lcv = cls1_reg_lcv.predict(x1_test)

print('R-square:', cls1_reg_lcv.score(x1_test, y1_test))
print('MSE:', mean_squared_error(y1_test,y1_test_predicted_lcv))
print('Coeffcients: ', cls1_reg_lcv.coef_)

### Cluster 2

In [ ]:
cls2_x = day7_cluster2.drop(columns=cols)
cls2_x = pd.get_dummies(cls2_x, columns = ['gaUserAcquisitionChannel','platform'], prefix=['Chan', 'Plat'], drop_first=False)
cls2_x = cls2_x.drop(columns = ['Plat_IOS','extraWords','invalidWords'])

var_heatmap2 = cls2_x
f, ax = plt.subplots(figsize=(20, 16))
corr = var_heatmap2.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax, annot=True)

In [ ]:
# LassoCV on cluster 2
cls2_x = preprocessing.normalize(cls2_x, norm='l2')
cls2_y = day7_cluster2['ad_Rev'] 

x2_train, x2_test, y2_train, y2_test = train_test_split(cls2_x, cls2_y, test_size=0.2, random_state=42)

reg_lcv = LassoCV(cv=5, random_state=0)
cls2_reg_lcv = reg_lcv.fit(x2_train, y2_train)

y2_test_predicted_lcv = cls2_reg_lcv.predict(x2_test)

print('R-square:',cls2_reg_lcv.score(x2_test, y2_test))
print('MSE:',mean_squared_error(y2_test,y2_test_predicted_lcv))
print('Coeffcients: ', cls2_reg_lcv.coef_)

### Cluster 3

In [ ]:
cls3_x = day7_cluster3.drop(columns=cols)
cls3_x = pd.get_dummies(cls3_x, columns = ['gaUserAcquisitionChannel','platform'], prefix=['Chan', 'Plat'], drop_first=False)
cls3_x = cls3_x.drop(columns = 'Plat_IOS')
cls3_x = cls3_x.drop(columns = ['extraWords','invalidWords','missionTime'])

var_heatmap3 = cls3_x
f, ax = plt.subplots(figsize=(20, 16))
corr = var_heatmap3.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax, annot=True)

In [ ]:
# LassoCV on cluster 3
cls3_x = preprocessing.normalize(cls3_x, norm='l2')
cls3_y = day7_cluster3['ad_Rev'] 
x3_train, x3_test, y3_train, y3_test = train_test_split(cls3_x, cls3_y, test_size=0.2, random_state=42)

reg_lcv = LassoCV(cv=5, random_state=0)
cls3_reg_lcv = reg_lcv.fit(x3_train, y3_train)

y3_test_predicted_lcv = cls3_reg_lcv.predict(x3_test)

print('R-square',cls3_reg_lcv.score(x3_test, y3_test))
print('MSE:', mean_squared_error(y3_test,y3_test_predicted_lcv))
print('Coeffcients: ', cls3_reg_lcv.coef_)

## KNN Regression to predict total ads revenue within each cluster

### Cluster 0

In [ ]:
n = np.arange(1, 300, 1)
parameters = {'n_neighbors': n}

neigh = KNeighborsRegressor()

knn_cv = GridSearchCV(neigh, parameters)
print(knn_cv.fit(x0_train, y0_train).best_params_['n_neighbors'])
cls0_neigh = KNeighborsRegressor(n_neighbors = knn_cv.fit(x0_train, y0_train).best_params_['n_neighbors']).fit(x0_train, y0_train)

y0_test_predicted_knn = cls0_neigh.predict(x0_test)

print('R-square:',cls0_neigh.score(x0_test, y0_test))
print('MSE: ', mean_squared_error(y0_test,y0_test_predicted_knn))

### Cluster 1

In [ ]:
n = np.arange(1, 100, 1)
parameters = {'n_neighbors': n}

neigh = KNeighborsRegressor()

knn_cv = GridSearchCV(neigh, parameters)
print(knn_cv.fit(x1_train, y1_train).best_params_['n_neighbors'])
cls1_neigh = KNeighborsRegressor(n_neighbors = knn_cv.fit(x1_train, y1_train).best_params_['n_neighbors']).fit(x1_train, y1_train)

y1_test_predicted_knn = cls1_neigh.predict(x1_test)

print('R-square:',cls1_neigh.score(x1_test, y1_test))
print('MSE: ', mean_squared_error(y1_test,y1_test_predicted_knn))

### Cluster 2

In [ ]:
n = np.arange(1, 100, 1)
parameters = {'n_neighbors': n}

neigh = KNeighborsRegressor()

knn_cv = GridSearchCV(neigh, parameters)
print(knn_cv.fit(x2_train, y2_train).best_params_['n_neighbors'])
cls2_neigh = KNeighborsRegressor(n_neighbors = knn_cv.fit(x2_train, y2_train).best_params_['n_neighbors']).fit(x2_train, y2_train)

y2_test_predicted_knn = cls2_neigh.predict(x2_test)

print('R-square:',cls2_neigh.score(x2_test, y2_test))
print('MSE: ', mean_squared_error(y2_test,y2_test_predicted_knn))

### Cluster 3

In [ ]:
n = np.arange(1, 100, 1)
parameters = {'n_neighbors': n}

neigh = KNeighborsRegressor()

knn_cv = GridSearchCV(neigh, parameters)
print(knn_cv.fit(x3_train, y3_train).best_params_['n_neighbors'])
cls3_neigh = KNeighborsRegressor(n_neighbors = knn_cv.fit(x3_train, y3_train).best_params_['n_neighbors']).fit(x3_train, y3_train)

y3_test_predicted_knn = cls3_neigh.predict(x3_test)

print('R-square:',cls3_neigh.score(x3_test, y3_test))
print('MSE: ', mean_squared_error(y3_test,y3_test_predicted_knn))